# TERM PROJECT
* TEAM NAME: ANS
* MEMBER NAMES: Abhiroop Kumar (ak56448), Nikhil Kumar (nk25627), Simoni K Dalal (skd939)
* DATABASE: DB_TEAM_ANS
* ROLE: ROLE_TEAM_ANS
* DUE: Dec 8 at 11:59pm

* FILE NAME: ANS_GOLD_VISUALIZATION.sql  

SOLUTION

In [ ]:
------------------------------------------------------------------
-- 0. SET CONTEXT
------------------------------------------------------------------

USE ROLE ROLE_TEAM_ANS;
USE WAREHOUSE ANIMAL_TASK_WH;
USE DATABASE DB_TEAM_ANS;
USE SCHEMA GOLD;


In [ ]:
------------------------------------------------------------------
-- 1. LANGUAGE INSTRUCTIONAL EFFORT (bar chart)
------------------------------------------------------------------
-- Gold visualization 1: Programming language instructional effort
-- This powers Visualization 1: “Total learning hours by programming language”.
------------------------------------------------------------------
SELECT
    LANGUAGE_NAME,
    COURSE_COUNT,
    TOTAL_TIME_HOURS,
    TOTAL_CHAPTERS,
    TOTAL_EXERCISES,
    TOTAL_VIDEOS
FROM G_LANGUAGE_INSTRUCTIONAL_EFFORT
ORDER BY TOTAL_TIME_HOURS DESC;


In [ ]:
------------------------------------------------------------------
-- 2. TRACK-LEVEL SUMMARY (bar chart with optional filter)
------------------------------------------------------------------
-- Gold visualization 2: Track-level content summary
-- This powers Visualization 2: “Total learning hours by track”.
-- (optional filter for 'Career vs Skill' in Streamlit)
------------------------------------------------------------------
SELECT
    TRACK_TITLE,
    IS_CAREER_FLAG,
    COURSE_COUNT,
    TOTAL_TIME_HOURS,
    TOTAL_CHAPTERS,
    TOTAL_EXERCISES,
    TOTAL_VIDEOS
FROM DB_TEAM_ANS.GOLD.G_TRACK_CONTENT_SUMMARY
ORDER BY TOTAL_TIME_HOURS DESC;


In [ ]:
------------------------------------------------------------------
-- 3. DIFFICULTY DISTRIBUTION (bar chart)
------------------------------------------------------------------
-- Gold visualization 3: Difficulty-level content and distribution
-- This powers Visualization 3: “Course distribution and content by difficulty level.”
------------------------------------------------------------------
SELECT
    DIFFICULTY_CODE,
    DIFFICULTY_ORDER,
    COURSE_COUNT,
    TOTAL_TIME_HOURS,
    TOTAL_CHAPTERS,
    TOTAL_EXERCISES,
    TOTAL_VIDEOS
FROM DB_TEAM_ANS.GOLD.G_DIFFICULTY_CONTENT_SUMMARY
ORDER BY DIFFICULTY_ORDER;


In [ ]:
##################################################################
## 4. STREAMLIT DASHBOARD FROM GOLD TABLES
##################################################################

# Import python packages
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session

# Get active Snowflake session (needed to run additional session.sql())
session = get_active_session()

# Convert SQL cell outputs to pandas DataFrames
# IMPORTANT: names must match the SQL cell "name" fields:
#   lang_effort_sql, track_content_sql, difficulty_content_sql
lang_df = lang_effort_sql.to_pandas()
track_df = track_content_sql.to_pandas()
diff_df = difficulty_content_sql.to_pandas()

# Basic Streamlit page configuration
st.title("DataCamp Learning Content – Gold Layer Dashboard")
st.caption(
    "Visualizations built from GOLD tables: "
    "`G_LANGUAGE_INSTRUCTIONAL_EFFORT`, "
    "`G_TRACK_CONTENT_SUMMARY`, "
    "`G_DIFFICULTY_CONTENT_SUMMARY`."
)

# ---- Tabs for three Gold layer visualizations ----
tab_lang, tab_track, tab_diff = st.tabs(
    ["1. By Programming Language", "2. By Track", "3. By Difficulty"]
)

# -------------------------------------------------------------------
# TAB 1: Programming Language Instructional Effort
# (Gold: G_LANGUAGE_INSTRUCTIONAL_EFFORT)
# -------------------------------------------------------------------
with tab_lang:
    st.subheader("Instructional Effort by Programming Language")

    # Optional: show data table
    st.dataframe(lang_df)

    # Main visualization: bar chart of total time by language
    st.markdown("**Total learning hours per programming language**")
    st.bar_chart(
        lang_df,
        x="LANGUAGE_NAME",
        y="TOTAL_TIME_HOURS"
    )

    # Secondary metric: total chapters or exercises (if you want)
    st.markdown("**Total chapters per programming language**")
    st.bar_chart(
        lang_df,
        x="LANGUAGE_NAME",
        y="TOTAL_CHAPTERS"
    )

# -------------------------------------------------------------------
# TAB 2: Track-level Summary
# (Gold: G_TRACK_CONTENT_SUMMARY)
# -------------------------------------------------------------------
with tab_track:
    st.subheader("Content Summary by Track")

    # Filter career vs skill tracks (IS_CAREER_FLAG assumed 1 = Career, 0 = Skill)
    track_type = st.radio(
        "Filter tracks by type:",
        options=["All", "Career Tracks", "Skill Tracks"],
        horizontal=True
    )

    track_filtered = track_df.copy()
    if track_type == "Career Tracks":
        track_filtered = track_filtered[track_filtered["IS_CAREER_FLAG"] == 1]
    elif track_type == "Skill Tracks":
        track_filtered = track_filtered[track_filtered["IS_CAREER_FLAG"] == 0]

    st.dataframe(track_filtered)

    st.markdown("**Total learning hours per track**")
    st.bar_chart(
        track_filtered,
        x="TRACK_TITLE",
        y="TOTAL_TIME_HOURS"
    )

    # Optional: show course count as a second chart
    st.markdown("**Number of courses per track**")
    st.bar_chart(
        track_filtered,
        x="TRACK_TITLE",
        y="COURSE_COUNT"
    )

# -------------------------------------------------------------------
# TAB 3: Difficulty Distribution & Depth
# (Gold: G_DIFFICULTY_CONTENT_SUMMARY)
# -------------------------------------------------------------------
with tab_diff:
    st.subheader("Course Distribution and Content by Difficulty")

    st.dataframe(diff_df)

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("**Course count by difficulty level**")
        st.bar_chart(
            diff_df,
            x="DIFFICULTY_CODE",
            y="COURSE_COUNT"
        )

    with col2:
        st.markdown("**Total learning hours by difficulty level**")
        st.bar_chart(
            diff_df,
            x="DIFFICULTY_CODE",
            y="TOTAL_TIME_HOURS"
        )
